In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping


2024-10-28 16:15:29.307340: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Path to the main directory containing "defective" and "good" subfolders
main_dir = '/Users/kuriankgeorge/Desktop/aws/Capstone/09_Fault_Findy/data/Digital images of defective and good condition tyres'

In [3]:
# Set image size and batch size
img_height, img_width = 160, 160  
batch_size = 32

In [4]:
# ImageDataGenerator with validation split
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,  
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # Reserve 20% for validation
)


In [5]:
# Train and validation generators
train_generator = data_gen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 1486 images belonging to 2 classes.


In [6]:
validation_generator = data_gen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 370 images belonging to 2 classes.


In [7]:
# Load MobileNetV2 with pretrained weights, excluding the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model layers initially


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [8]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [9]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [11]:
# Train the model
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping]
)

Epoch 1/10


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-10-28 16:18:03.790449: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3 of 8
2024-10-28 16:18:21.142516: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


46/46 ━━━━━━━━━━━━━━━━━━━━ 325s 6s/step - accuracy: 0.5063 - loss: 1.0575 - val_accuracy: 0.7727 - val_loss: 0.4957
Epoch 2/10
 1/46 ━━━━━━━━━━━━━━━━━━━━ 40s 909ms/step - accuracy: 0.7500 - loss: 0.6127

2024-10-28 16:23:09.068180: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.7500 - loss: 0.6127 - val_accuracy: 0.7222 - val_loss: 0.5686
Epoch 3/10


...

46/46 ━━━━━━━━━━━━━━━━━━━━ 219s 4s/step - accuracy: 0.8298 - loss: 0.3679 - val_accuracy: 0.7756 - val_loss: 0.4505
Epoch 8/10
 1/46 ━━━━━━━━━━━━━━━━━━━━ 28s 644ms/step - accuracy: 0.8750 - loss: 0.2539

2024-10-28 16:34:56.206724: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8750 - loss: 0.2539 - val_accuracy: 0.6667 - val_loss: 0.5614
Epoch 9/10


2024-10-28 16:34:57.566482: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-28 16:35:10.893896: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8
2024-10-28 16:35:23.836522: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


46/46 ━━━━━━━━━━━━━━━━━━━━ 203s 4s/step - accuracy: 0.8411 - loss: 0.3678 - val_accuracy: 0.7983 - val_loss: 0.4150


In [12]:
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 20  # Unfreeze the last 20 layers for fine-tuning

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [13]:
# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
# Fine-tune the model
fine_tune_epochs = 10
total_epochs = epochs + fine_tune_epochs

In [15]:
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=[early_stopping]
)

Epoch 9/20


2024-10-31 08:23:35.731360: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 2 of 8
2024-10-31 08:23:50.000553: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 4 of 8
2024-10-31 08:24:02.039676: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 6 of 8
2024-10-31 08:24:10.684706: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


46/46 ━━━━━━━━━━━━━━━━━━━━ 405s 7s/step - accuracy: 0.7056 - loss: 0.6161 - val_accuracy: 0.8011 - val_loss: 0.4313
Epoch 10/20
 1/46 ━━━━━━━━━━━━━━━━━━━━ 36s 803ms/step - accuracy: 0.8438 - loss: 0.4037

2024-10-31 08:29:36.111674: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.8438 - loss: 0.4037 - val_accuracy: 0.7222 - val_loss: 0.5150
Epoch 11/20


2024-10-31 08:29:37.236687: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-31 08:29:47.867981: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 3 of 8
2024-10-31 08:29:58.693639: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:21: Filling up shuffle buffer (this may take a while): 6 of 8
2024-10-31 08:30:06.050440: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


46/46 ━━━━━━━━━━━━━━━━━━━━ 197s 4s/step - accuracy: 0.7639 - loss: 0.5450 - val_accuracy: 0.8011 - val_loss: 0.4272


In [16]:
model.save('faultfindy_tire_classification_model_mobilenet.h5')

In [17]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy after fine-tuning: {accuracy:.2f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.7507 - loss: 0.4254
Validation accuracy after fine-tuning: 0.77
